In [469]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    v = fp.read()

def removestopwords(line):
    #print("original line",line)
    querywords = line.split()

    resultwords  = [word for word in querywords if word.lower() not in v]
    result = ' '.join(resultwords)
    #print(result)
    return result

In [470]:
import math
import re
from collections import Counter
WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [463]:
import nltk
nltk.download('indian')
nltk.download('punkt')

[nltk_data] Downloading package indian to C:\Users\HM-
[nltk_data]     AM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\indian.zip.
[nltk_data] Downloading package punkt to C:\Users\HM-
[nltk_data]     AM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [491]:
import nltk
from nltk.tag import tnt 
from nltk.corpus import indian 

#train_data = indian.tagged_sents('gu.pos')
train_data = indian.tagged_sents('C:/Users/HM-AM/Documents/Jupyter Notebooks/gu.pos')
tnt_pos_tagger = tnt.TnT() 
tnt_pos_tagger.train(train_data)

In [492]:
size = int(len(train_data) * 0.9)

In [493]:
#train_sents = train_data
train_sents = train_data[:size]
test_sents = train_data[size:]

In [494]:
def backoff_tagger(train_sentences, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sentences, backoff=backoff)
        
    return backoff

In [495]:
back_tagger = nltk.DefaultTagger('NN')
Combine_tagger = backoff_tagger(train_sents,
[nltk.UnigramTagger, nltk.BigramTagger, nltk.TrigramTagger], backoff = back_tagger)

Combine_tagger.evaluate(test_sents)

0.8358101438061951

In [471]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [510]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize

#variation 1
# keywords=['N-NNP','N_NN','JJ','N_NNV','N_NST','QT_QTC','QT_QTF','QT_QTO','PSP']
# keywords1=['V_VAUX','RP_RPD','V_VM','V_VM_VF','VM_VM_VINF','V_VM_VNF','V_VM_VNG','PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ',
#           'RB']
# #variation 2
# keywords=['N-NNP','N_NN','N_NNV','N_NST','PSP','JJ','RB','V_VAUX','V_VM','V_VM_VF',
#           'VM_VM_VINF','V_VM_VNF','V_VM_VNG']
# keywords1=['RP_RPD','PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ','QT_QTC','QT_QTF','QT_QTO']
#variation 3
# keywords=['N-NNP','N_NN','N_NNV','N_NST','PSP','JJ','RB','NN','QT_QTC','QT_QTF','QT_QTO']
# keywords1=['RP_RPD','PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ','V_VAUX','V_VM','V_VM_VF',
#           'VM_VM_VINF','V_VM_VNF','V_VM_VNG']
# # # variation 4
# keywords=['NN','QT_QTC','QT_QTF','QT_QTO','RP_RPD','RP_INTF','JJ','RB',
#          'PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ','N-NNP','N_NN','N_NNV','N_NST','PSP']
# keywords1=['V_VAUX','V_VM','V_VM_VF','VM_VM_VINF','V_VM_VNF','V_VM_VNG','DM_DMD','RD_PUNC','CC_CCD','CC_CCS']
# #variation 5
# keywords=['N-NNP','N_NN','JJ','N_NNV','N_NST','QT_QTC','QT_QTF','QT_QTO','PSP','RB','CC_CCD','CC_CCS','DM_DMD','NN']
# keywords1=['V_VAUX','RP_RPD','V_VM','V_VM_VF','VM_VM_VINF','V_VM_VNF','V_VM_VNG','PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ']
#variation 6
keywords=['N-NNP','N_NN','N_NNV','N_NST','PSP','JJ','NN']
keywords1=['RP_RPD','RP_INJ','RP_INTF','PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ','RB','V_VAUX','V_VM','V_VM_VF',
          'VM_VM_VINF','V_VM_VNF','V_VM_VNG','DM_DMD','DM_DMR','DM_DMQ','CC_CCD','CC_CCS']
fc=[]
li=[]
maintext=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file8(filename,createfile):
    tokenize_words=[]
    titlesentences=[]
    file1 = open(createfile, "w", encoding="utf-8")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    docno=soup.find_all('docno')
    text=soup.find_all('text')
    titleno = soup.find_all('title')
    instances = {}
    my_id = 0
    for n,tit,maintitle in zip(docno,text,titleno):
        count=0
        tit=tit.get_text()
        tit=str(tit)
        #print("this is text",tit)
        maintitle=maintitle.get_text()
        maintitle=str(maintitle)
        maintitlelist=[]
        for tokenize in indic_tokenize.trivial_tokenize(maintitle):
            titlesentences.append(tokenize)
        list8 = [''.join(c for c in s if c not in string.punctuation) for s in titlesentences]
        list8 = [s for s in list8 if s]
        originalsen=[]
        sentences=sentence_tokenize.sentence_split(tit, lang='gu')
        #print(sentences)
        list3 = [''.join(c for c in s if c not in string.punctuation) for s in sentences]
        list3 = [s for s in list3 if s]
        #list3 = [s for s in sentences if s]
        for t in list3:
            sen=removestopwords(t)
            #sen=t
            #matching with title
            if(len(sen)==0):
                sen=sen+"hi"
            querywords = sen.split()
            h = len(querywords)
            while(h>=0):
                if(h==len(querywords)):
                    h=h-1
                    
                for x in range(len(list8)):
#                     print(list3[x])
#                     print(querywords[j])
                    vector1 = text_to_vector(list8[x])
                    vector2 = text_to_vector(querywords[h])
                    val = get_cosine(vector1, vector2)
                    if(val>=1):
                        #print(list8[x]+"matching"+querywords[h])
                        count=count+1
                        #print("this is count",count)
                #i=i+1
                h=h-1
            tagged_words = (Combine_tagger.tag(nltk.word_tokenize(sen)))
            #originalsen.append(sen)
        
            lst1=[]
            for w in tagged_words:
                lst1.append(w[1])


            b = nltk.FreqDist()
            i=1
#             #print("this is count",count)
#             i=i+count
            j=1
            #j=j-count
            for w in lst1:
                if w in keywords:
                    i=i+1

            for w1 in lst1:
                if w1 in keywords1:
                    j=j+1
            
            
            f=(i-j+100)/2
            f=f+count
            fc.append(f)
            
        for i in range(len(fc)):
            li.append([fc[i],i])
        li.sort(reverse=True)
        #print("this is li",li)
        
        sort_index = []
        for x in li:
            sort_index.append(x[1])
        #print(sort_index)
        compressionratio = round((len(sort_index)*40)/100)
        #print("this is compression ratio",compressionratio)
        newsen=[]
        for j in range(compressionratio):
            newsen.append(sentences[sort_index[j]])
        #print("this is newsen",newsen)    
        originalfull_str = convert_list_to_string(newsen)
        originalfull_str=" ".join(originalfull_str.split())
        n=str(n)
        file1.write(originalfull_str)
        fc.clear()
        li.clear()
#         for i in range(len(fc)):
#             li.append([fc[i],i])
#         li.sort(reverse=True)
#         sort_index = []
#         for x in li:
#             sort_index.append(x[1])
        #print(sort_index)
        
                
        instances = {}
    
        
    return instances

In [511]:
import os

#for root, dirs, files in os.walk("/home/harsh/Documents/gujdocs/gsf2003/"):
for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization dataset/health/"):    
#for root, dirs, files in os.walk("G:/Movies/post_1/"):    
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file8(os.path.join(root, file),"G:/Movies/gujarati Text summarization datasetformal/health/"+file)
                #load_file8(os.path.join(root, file),"G:/Movies/post_1_summary/"+file)
#average = Average(fc)
#print(average)

In [512]:
from rouge.rouge import rouge_n_sentence_level

In [513]:
from rouge.rouge import rouge_n_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level

In [514]:
from statistics import mean
def Average(lst):
    return mean(lst)

In [515]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
rouge1=[]
rouge2=[]
rougel=[]
rougew=[]

class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file1(filename,reference):
    tokenize_words=[]
    tokenize_words1=[]
    
    #file1 = open(reference, "r")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    #print(data)
    
    for tokenize in indic_tokenize.trivial_tokenize(data):
            tokenize_words.append(tokenize)
        # Convert list of strings to string
    originalfull_str = convert_list_to_string(tokenize_words)
    senlateron=[]
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    i=1
    with codecs.open(reference+"a1_"+str(i)+".txt", encoding='utf-8') as f1:
        data1 = f1.read()
    i=i+1
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    senlateron1=[]
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
    rouge1.append(rouge_1)
    #print('ROUGE-1: %f' % rouge_1)
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
    #print('ROUGE-2: %f' % rouge_2)
    rouge2.append(rouge_2)
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-L: %f' % rouge_l)
    rougel.append(rouge_l)
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-W: %f' % rouge_w)
    rougew.append(rouge_w)
    print(rouge1)
    average1 = Average(rouge1)
    print("average of rouge1",average1)
    print(rouge2)
    average2 = Average(rouge2)
    print("average of rouge2",average2)
    print(rougel)
    averagel = Average(rougel)
    print("average of rougel",averagel)
    print(rougew)
    averagew = Average(rougew)
    print("average of rougew",averagew)
   
    instances = {}


    return instances

In [516]:
import os
for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization datasetformal/health/"):    
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file1(os.path.join(root, file),"C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/health/annotator1/")

[0.884886452623336]
average of rouge1 0.884886452623336
[0.7931034482758622]
average of rouge2 0.7931034482758622
[0.884886452623336]
average of rougel 0.884886452623336
[0.30324260905252237]
average of rougew 0.30324260905252237
[0.884886452623336, 0.6078316773816481]
average of rouge1 0.746359065002492
[0.7931034482758622, 0.3779988297249854]
average of rouge2 0.5855511390004238
[0.884886452623336, 0.6078316773816481]
average of rougel 0.746359065002492
[0.30324260905252237, 0.18929283859264426]
average of rougew 0.24626772382258333
[0.884886452623336, 0.6078316773816481, 0.8084428514536042]
average of rouge1 0.767053660486196
[0.7931034482758622, 0.3779988297249854, 0.4814667198086887]
average of rouge2 0.5508563326031788
[0.884886452623336, 0.6078316773816481, 0.8076463560334528]
average of rougel 0.7667881620128123
[0.30324260905252237, 0.18929283859264426, 0.26957849834278197]
average of rougew 0.2540379819959829
[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208]
average of rouge1 0.7056121344612896
[0.7931034482758622, 0.3779988297249854, 0.4814667198086887, 0.22832980972515857, 0.48012976480129765, 0.41208791208791207, 0.32775119617224885, 0.3811957077158916, 0.3892078071182549, 0.42359249329758714, 0.4032567049808429, 0.4433164128595601, 0.40320855614973267, 0.46642747309072274]
average of rouge2 0.4293623454149104
[0.884886452623336, 0.6078316773816481, 0.8076463560334528, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7085247575293516, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.6591549295774647, 0.7019230769230769, 0.7885304659498208]
average of rougel 0.7053255229543368
[0.30324260905252237, 0.18929283859264426, 0.26957849834278197, 0.1049

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272]
average of rouge1 0.7088379839995795
[0.7931034482758622, 0.3779988297249854, 0.4814667198086887, 0.22832980972515857, 0.48012976480129765, 0.41208791208791207, 0.32775119617224885, 0.3811957077158916, 0.3892078071182549, 0.42359249329758714, 0.4032567049808429, 0.4433164128595601, 0.40320855614973267, 0.46642747309072274, 0.43107634205557044, 0.46672504378283713, 0.4791987673343606, 0.41687344913151364, 0.39128397375820057, 0.3876556578234976]
average of rouge2 0.42919430348473625
[0.884886452623336, 0.6078316773816481, 0.8076463560334528, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.60334528076

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272, 0.7540500736377025, 0.8219402374130167, 0.6705882352941177, 0.8560070671378092, 0.7550607287449392]
average of rouge1 0.7213762408887671
[0.7931034482758622, 0.3779988297249854, 0.4814667198086887, 0.22832980972515857, 0.48012976480129765, 0.41208791208791207, 0.32775119617224885, 0.3811957077158916, 0.3892078071182549, 0.42359249329758714, 0.4032567049808429, 0.4433164128595601, 0.40320855614973267, 0.46642747309072274, 0.43107634205557044, 0.46672504378283713, 0.4791987673343606, 0.41687344913151364, 0.39128397375820057, 0.3876556578234976, 0.40393120393120385, 0.47931175747644406, 0.40396530359355637, 0.500

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272, 0.7540500736377025, 0.8219402374130167, 0.6705882352941177, 0.8560070671378092, 0.7550607287449392, 0.7498267498267498, 0.6723068245910886, 0.7347593582887701, 0.4958782498414711]
average of rouge1 0.7133509380954226
[0.7931034482758622, 0.3779988297249854, 0.4814667198086887, 0.22832980972515857, 0.48012976480129765, 0.41208791208791207, 0.32775119617224885, 0.3811957077158916, 0.3892078071182549, 0.42359249329758714, 0.4032567049808429, 0.4433164128595601, 0.40320855614973267, 0.46642747309072274, 0.43107634205557044, 0.46672504378283713, 0.4791987673343606, 0.41687344913151364, 0.39128397375820057, 0.387655

...

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272, 0.7540500736377025, 0.8219402374130167, 0.6705882352941177, 0.8560070671378092, 0.7550607287449392, 0.7498267498267498, 0.6723068245910886, 0.7347593582887701, 0.4958782498414711, 0.7067099567099567, 0.4902200488997554, 0.8509615384615385, 0.5909631391200951, 0.7161664033701947, 0.5691747572815534, 0.6569178852643419, 0.4067796610169492, 0.7572614107883818, 0.803951367781155, 0.8257088846880908, 0.7813712807244502, 0.462853385930309, 0.7348242811501599, 0.8556876061120543, 0.8346755092373284, 0.4101161995898839, 0.6736958934517202, 0.8326289095519864, 0.8261238337574216, 0.640922190201729, 0.7271756540309664, 

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272, 0.7540500736377025, 0.8219402374130167, 0.6705882352941177, 0.8560070671378092, 0.7550607287449392, 0.7498267498267498, 0.6723068245910886, 0.7347593582887701, 0.4958782498414711, 0.7067099567099567, 0.4902200488997554, 0.8509615384615385, 0.5909631391200951, 0.7161664033701947, 0.5691747572815534, 0.6569178852643419, 0.4067796610169492, 0.7572614107883818, 0.803951367781155, 0.8257088846880908, 0.7813712807244502, 0.462853385930309, 0.7348242811501599, 0.8556876061120543, 0.8346755092373284, 0.4101161995898839, 0.6736958934517202, 0.8326289095519864, 0.8261238337574216, 0.640922190201729, 0.7271756540309664, 

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272, 0.7540500736377025, 0.8219402374130167, 0.6705882352941177, 0.8560070671378092, 0.7550607287449392, 0.7498267498267498, 0.6723068245910886, 0.7347593582887701, 0.4958782498414711, 0.7067099567099567, 0.4902200488997554, 0.8509615384615385, 0.5909631391200951, 0.7161664033701947, 0.5691747572815534, 0.6569178852643419, 0.4067796610169492, 0.7572614107883818, 0.803951367781155, 0.8257088846880908, 0.7813712807244502, 0.462853385930309, 0.7348242811501599, 0.8556876061120543, 0.8346755092373284, 0.4101161995898839, 0.6736958934517202, 0.8326289095519864, 0.8261238337574216, 0.640922190201729, 0.7271756540309664, 

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272, 0.7540500736377025, 0.8219402374130167, 0.6705882352941177, 0.8560070671378092, 0.7550607287449392, 0.7498267498267498, 0.6723068245910886, 0.7347593582887701, 0.4958782498414711, 0.7067099567099567, 0.4902200488997554, 0.8509615384615385, 0.5909631391200951, 0.7161664033701947, 0.5691747572815534, 0.6569178852643419, 0.4067796610169492, 0.7572614107883818, 0.803951367781155, 0.8257088846880908, 0.7813712807244502, 0.462853385930309, 0.7348242811501599, 0.8556876061120543, 0.8346755092373284, 0.4101161995898839, 0.6736958934517202, 0.8326289095519864, 0.8261238337574216, 0.640922190201729, 0.7271756540309664, 

[0.884886452623336, 0.6078316773816481, 0.8084428514536042, 0.35045742434905, 0.8265802269043759, 0.694840834248079, 0.6033452807646356, 0.7095456865747831, 0.6662844036697249, 0.78125, 0.7933014354066985, 0.660281690140845, 0.7029914529914529, 0.7885304659498208, 0.6104071178215152, 0.8442694663167103, 0.821401077752117, 0.7801652892561984, 0.5302107728337236, 0.711736073553272, 0.7540500736377025, 0.8219402374130167, 0.6705882352941177, 0.8560070671378092, 0.7550607287449392, 0.7498267498267498, 0.6723068245910886, 0.7347593582887701, 0.4958782498414711, 0.7067099567099567, 0.4902200488997554, 0.8509615384615385, 0.5909631391200951, 0.7161664033701947, 0.5691747572815534, 0.6569178852643419, 0.4067796610169492, 0.7572614107883818, 0.803951367781155, 0.8257088846880908, 0.7813712807244502, 0.462853385930309, 0.7348242811501599, 0.8556876061120543, 0.8346755092373284, 0.4101161995898839, 0.6736958934517202, 0.8326289095519864, 0.8261238337574216, 0.640922190201729, 0.7271756540309664, 

text=""" વાઈવા આપતી વખતે, એપ્રનના પહેલા બે બટન ખોલી નાંખતી હશે. એમબીબીએસમાં મારી જ સાથે ભણતા એક મેડિકલ સ્ટુડન્ટે અમારા ક્લાસની ટોપર અને સૌથી હોંશિયાર છોકરી વિશે કરેલી ટિપ્પણી મને આજે પણ યાદ છે. આપણી આસપાસ એવા અઢળક લોકો છે જે એવું માને છે કે સફળતાનો રસ્તો પુરુષ કરતા સ્ત્રી માટે વધારે ટૂંકો હોય છે.  
પુરુષની સફળતાને એના સાહસ, સમર્પણ અને મહેનત સાથે જોડવામાં આવે છે  જ્યારે  સ્ત્રીની સફળતાને શક્યતાઓ સાથે. એ કોઈ કલાકાર હોય, જર્નલીસ્ટકે ઓફિસ વર્કર. કોઈપણ ક્ષેત્રમાં એક સ્ત્રી જ્યારે અચાનક લાઈમ-લાઈટમાં આવે છે, ત્યારે એની પ્રસિદ્ધિનો યશ એની પ્રતિભા કરતા એની સુંદરતા, એના ફિઝીક અને એની 'ઉદારતા' ને  આપવામાં આવે છે. 
સફળતાની સીડીઓ ચડી રહેલી સ્ત્રીને પાછળ ધકેલવા અને એણે મેળવેલી સિદ્ધિઓનો અનાદર કરવા માટે અન્ય કોઈ સ્ત્રી કે પુરુષ પાસે રહેલું  સૌથી  જુનું અને ધારદાર  હથિયાર એટલે  'કેરેક્ટર એસેસીનેશન.' એ ઈર્ષા હોય  કે ઈન્ફીરીયોરીટી, રેસમાં રહેલી વ્યક્તિને જ્યારે એવું પ્રતીત થાય છે કે આ સ્ત્રીને હવે 'મેરીટ' થી નહીં હરાવી શકાય ત્યારે વાત ચારિત્ર્ય પર આવે છે.
આ ફેમિનિઝમ નથી. કારણકે ફેમિનિઝમ તો સ્ત્રીઓને સમાન હક અને તક અપાવવા માટે હોય છે. ફેમિનિઝમ સ્ત્રીઓને લિબરેટ કરવા માટે હોય છે. જ્યારે આ સમસ્યા તો સ્ત્રીઓના લિબરેશન પછીની છે. 
ફેમિનિઝમના સહારે અને ભરોસે રહેવાને બદલે એક સ્ત્રી જ્યારે પોતાના હકની સિદ્ધિ અને સફળતા, પોતાની મહેનત અને ટેલેન્ટના જોર પર ઝૂંટવી લે છે ત્યારે વાત એના ચારિત્ર્ય પર આવે છે. 
ઈસ્ટ્રોજન સામેની દોડમાં જ્યારે ટેસ્ટોસ્ટેરોન ઘવાય છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે. આજ સુધી ફક્ત 'સુંદરતા' ને જ એક સ્ત્રીની USP ગણનારા સમાજની આંખો જ્યારે એની બુદ્ધિ અને પ્રતિભાથી અંજાઈ છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે.
 સ્ત્રીના સક્સેસને એની સેક્સ્યુઆલીટી સાથે જોડી દેવામાં આવે છે. 
સ્ત્રીની સફળતા 'સામાજિક અપચો' સર્જે છે. આપણે ત્યાં પોલીટીક્સ કે ક્રિકેટ કરતા પણ વધારે કશુંક ચર્ચાતું હોય, તો એ એક સ્ત્રીની સફળતા છે. દરેક પાસે પોતાના અભિપ્રાયો અને તારણો છે. 
જે સમયે કોઈપણ જાતના જજમેન્ટ પાસ કર્યા વગર એક આગળ વધી રહેલી સ્ત્રીની સફળતાને આપણે 'એઝ ઈટ ઈઝ' સ્વીકારતા થઈ જશું, ત્યારે આપણે સિવિલાઈઝ્ડ કહેવાશું. ખ્યાતિ, લોકપ્રિયતા કે પ્રસિદ્ધિ મેળવવા માટે દરેક સ્ત્રીએ 'કોમ્પ્રોમાઈઝ' કરવું જ પડે છે,  એવું માનનારા લોકો પોતાની સમજણ સાથે કોમ્પ્રોમાઈઝ કરી રહ્યા છે. સ્ત્રીની સફળતા એણે કરેલું સમાધાન નથી, એણે કરેલી સાધના છે. 
હા, એણે કશુંક ઓફર કર્યું છે. પોતાની મહેનત, પ્રતિભા, સમર્પણ અને સાહસ. એણે કોમ્પ્રોમાઈઝ પણ કર્યું છે. કુટુંબને આપેલા સમય સાથે, પોતાના મોજ-શોખ અને સુખ-સગવડો સાથે.
એક સ્ત્રી જ્યારે પોતાના 'કમ્ફોર્ટ ઝોન' માંથી બહાર નીકળે છે, ત્યારે એની સફળતા બીજા અનેક લોકો માટે 'અનકમ્ફોર્ટેબલ' બની જતી હોય છે. 
-ડૉ. નિમિત્ત ઓઝ"""

In [15]:
text="""All said and done, મને મળી રહેલા પર્સનલ મેસેજીસની સંખ્યા 'અનમેનેજબલ' થઈ રહી છે. મેઈલ કે મેસેજ દ્વારા જે રીતે અને જે સંખ્યામાં લોકો પોતાની ઉદાસી કબુલી રહ્યા છે, પોતાને બચાવવા માટે રીક્વેસ્ટ કરી રહ્યા છે, એ જોઈને હું સ્તબ્ધ છું. મેં ક્યારેય નહોતું વિચાર્યું કે સોશિયલ મીડિયા પર 'ફીલીંગ હેપ્પી' કે 'ફિલિંગ એક્સાઈટેડ' લખીને પોતાના હસતા ફોટોઝ શેર કરનારા લોકો અંદરથી આટલા બધા ઉદાસ હશે. 
So I thought, Let me share something from my personal experience.  કદાચ કોઈને ઉપયોગી થાય. 
૧. સૌથી પહેલા તાત્કાલિક કોઈ થેરાપીસ્ટ, મનોચિકિત્સક કે કાઉન્સેલરને મળો. સારવારનો પહેલો તબક્કો સ્વીકાર છે. 'હા, છીએ ઉદાસ' એવું વટથી જ્યાં સુધી સ્વીકારશું નહીં, ત્યાં સુધી એમાંથી બહાર નહીં આવી શકાય. ASAP  દવા કે કાઉન્સેલિંગ સેશન શરૂ કરો. ડિપ્રેશનની દવા અસર કરતા ૧૫-૨૦ દિવસ લાગે છે. ત્યાં સુધી 'નો સ્યુસાઈડ કોન્ટ્રેક્ટ' સાઈન કરો. તમારી જાતને અને થેરાપીસ્ટને પ્રોમીસ કરો કે 'ક્ષણિક ઉદાસીનો ભોગ બનીને તમે મૂર્ખામીભર્યું કોઈ કાયમી પગલું નહીં ભરો.' યાદ રાખો, મૃત્યુ ઈરરીવર્સીબલ છે. 
૨. 'મેન્ટલ ડાયેરીયા' બંધ કરવા માટે મગજને બિનજરૂરી અને વાહિયાત માહિતીઓ પીરસવાનું બંધ કરો. એન્ડ ઈન પ્રોસેસ ઓફ ડૂઇંગ સો, સૌથી પહેલા સોશિયલ મીડિયા અને ન્યુઝ ચેનલ્સ બંધ કરો. ફેસબુક, ઇન્સ્ટા, વ્હોટ્સ-એપ પરથી થોડો સમય બ્રેક લઈ લો. સોશિયલ મીડિયા એ ડિપ્રેશનનું સૌથી મોટું કારણ છે કારણકે Every time we sign into it, unknowingly we are landing into the zone of comparison. કોઈપણ સ્તરે 'કમ્પેરીઝન' આપણા માનસિક સ્વાસ્થ્ય માટે જીવલેણ છે. 
સંપર્ક માટેના ત્રણ શ્રેષ્ઠ રસ્તાઓ જ ખુલ્લા રાખો. 'રૂટીન પ્રાયોરીટી' કામ હોય તો ઈ-મેઈલ, સેમી-અરજન્ટ હોય તો ટેક્સ્ટ મેસેજ (SMS) અને બહુ અરજન્ટ હોય તો ફોન કોલ. 
સોશિયલ મીડિયા પરથી બ્રેક લેવાથી આપણે કશું જ ગુમાવવાના નથી. આપણે કોઈ અગત્યની વસ્તુ કે વ્યક્તિને મિસ નહીં કરીએ. એન્ડ બીલીવ મી, નો-બડી ઇઝ ગોઈંગ ટુ મિસ અસ ઓલ્સો. (I took a break for 3 months from all social media last year). એ દરમિયાન જે માનસિક શાંતિનો અનુભવ થાય છે, એ પ્રાઈસ-લેસ છે. ટ્રાય ઇટ યોરસેલ્ફ.  
બ્રેક ન લો તો પણ, રીસર્ચ એવું કહે છે કે સોશિયલ મીડિયા પર ૩૦ મીનીટ કે ૧ કલાકથી વધારે સમય ન પસાર કરો. It will cause you depression. 
૩. મેડિટેશન. મેડિટેશન. મેડિટેશન. આજ નહીં તો કાલ નહીં તો વર્ષો પછી, મેડિટેશન સિવાય ઉદ્ધાર નથી. We all are constantly surrounded by people full of shit, and only Meditation can rescue us. 
પ્રોબ્લેમ એ છે કે આપણે જેટલા યુદ્ધને ગ્લોરીફાય કર્યા છે, એટલા બુદ્ધને કર્યા હોત તો આટલું બધું હ્યુમન સફરીંગ ન હોત. 
મેડિટેશન અંગે તમારા મનમાં જેટલી ખોટી માન્યતાઓ હોય, એટલી કાઢી નાખો. મેડિટેશન એ થોટલેસનેસ નથી. તમારે કોઈ શીબીર કે ધ્યાન ગુરુ પાસે જવાની જરૂર નથી. કોઈ જંગલમાં કે પર્વત પર જવાની જરૂર નથી. એ તમે ઘેર બેઠા કરી શકો છો અને એ પણ એકલા. 
'ગાઇડેડ મેડિટેશન' માટે તમારા ફોનમાં 'Calm' કે 'Headspace' એપ ડાઉનલોડ કરો. (હું હેડસ્પેસ યુઝ કરું છું એન્ડ આઈ એમ લવીંગ ઇટ.'
મેડિટેશન અંગે બેઝીક્સ જાણવું હોય તો આટલું વાચી જાવ. ધિસ વિલ હેલ્પ યુ અ લોટ. https://www.thinklessandgrowrich.com/meditation-for-beginners-the-complete-guide/
૪. પુસ્તકો વાચો. Your brain deserves much better content than non-sense forwards, silly jokes, and good morning messages.  Few good reads are : 
1. Kick the thinking habit by Richard Paterson 
2. The subtle art of not giving a F*ck  by Mark Manson 
3.  The power of Now by Eckhart Tolle 
4.  Any book by Dalai Lama
5.  Any book by Mahatma Gandhi 
6.  Any book on Buddha and Buddhism 
7.  વિનોબાજીના ગીતા પ્રવચનો 
(ફોનમાંથી નકામી એપ ડીલીટ કરીને એમેઝોન કિન્ડલ ઈન્સ્ટોલ કરો. See, How life changes !)
૫. ઈકો-થેરાપી : જંગલ, પર્વત, નદી, દરિયો અને ખુલ્લું આકાશ. આ સૌથી મોટા એન્ટી-ડીપ્રેસન્ટ્સ છે. Nature heals the soul.  આટલે બધે દૂર ન જઈ શકાય, તો ઘરની આસપાસ રહેલા ગાર્ડન કે પાર્કમાં જાવ. વૃક્ષો સાથે સમય વિતાવો. ક્યારેય ડિપ્રેશન નહીં આવે. હવે, તો આખી ફોરેસ્ટ મેડિસિનની બ્રાંચ ડેવલપ થઈ રહી છે, જે કહે છે કે ‘If you hug a tree, It will make you happy.'  (https://qz.com/1208959/japanese-forest-medicine-is-the-art-of-using-nature-to-heal-yourself-wherever-you-are/)
૬. મનમાં આવતા નેગેટીવ, નિરાશાજનક અને વાહિયાત વિચારોનું ઉત્સર્જન કરો. ડિપ્રેશન એ બીજું કાંઈ નથી પણ અવ્યક્ત રહી ગયેલા એટેન્શન સીકિંગ વિચારોનો આપણી બુદ્ધિ સામે થયેલો બળવો છે. કેથાર્સીસ કે વિરેચન માટેનું કોઈ એક માધ્યમ શોધી કાઢો. બેસ્ટ વે ઈઝ ટુ રાઈટ ડાઉન યોર ફીલિંગ્સ, ઇમોશન્સ, ફીઅર્સ એન્ડ ઈનસિક્યોરીટીઝ. 
દરેક નબળા અને હેરાન કરતા વિચારોની કાગળ પર ઉલટી કરી નાખો. પછી એ કાગળ ફાડી કે બાળી નાખો. It works ! 
૭. ગ્રેટીટ્યુડ ટેકનીક : રોજ સવારે ઉઠીને સૌથી પહેલા એક કાગળ પર એવી ત્રણ વસ્તુઓ લખો, જેના માટે તમે ઈશ્વરના આભારી છો. એ કાંઈ પણ હોય શકે છે. જેમકે તમે જીવો છો, તમારું ફેમીલી સ્વસ્થ છે, તમારા સંતાનો સુખી છે, તમારા ઘરની ઉપર છત છે, ઘરમાં ખાવાનું છે, એની ડેમ થીંગ.
એક ગ્લોબલ પોવર્ટી ફેક્ટ : આ વિશ્વની અડધી જનસંખ્યા એટલે કે ૩ અબજથી વધારે લોકો રોજના ૨.૫ ડોલર (એટલે કે બસોથી પણ ઓછા રૂપિયામાં) પોતાનું ગુજરાન ચલાવે છે. આટલા રૂપિયાનું તો આપણું વાહન દરરોજનું પેટ્રોલ પી જતું હશે. 
યુનિસેફના આંકડા પ્રમાણે વિશ્વના ૨૨,૦૦૦ બાળકો દરરોજ ભૂખમરાથી મૃત્યુ પામે છે.  
આવું બધું વિચારશું, તો મરવાની ઈચ્છા નહીં થાય. 
૮. ભલે આળસ આવે, પણ પરાણે કસરત કરો. એક્સરસાઈઝનો સૌથી મોટો ફાયદો માનસિક સ્વાસ્થ્ય પર થાય છે. જીમ કરો, સાયકલીંગ, રનીંગ, સ્કીપીંગ, સ્વીમીંગ કાંઈ પણ કરો. કાંઈ નહીં તો ગો ઓન અ લોંગ વોક. એ કરવાથી શરીરમાં એન્ડોર્ફીન્સ અને ડોપામીન રીલીઝ થાય છે. ધિસ વિલ મેક યુ ફીલ ગુડ. 
૯. મ્યુઝીક આત્માનો ખોરાક છે. હાવ કેન યુ ઈગ્નોર ધેટ ? તમારા હેડ-ફોન્સ લગાડો અને તમારા ગમતા પ્લે-લીસ્ટમાં ખોવાઈ જાવ.
૧૦. ધ લીસ્ટ ઈઝ એન્ડ-લેસ. હજી ઘણું બાકી છે. આઈ પ્લાન ટુ સ્ટાર્ટ અ સીરીઝ ઓફ આર્ટીકલ્સ, પોડકાસ્ટ એન્ડ વિડિયોઝ ઓન મેન્ટલ હેલ્થ. ઈફ યુ આર રીઅલી ઈન્ટરેસ્ટેડ, મારા બ્લોગ https://drnimitoza.in/ પર જઈને સબસ્ક્રાઈબ કરો. 
યાદ રાખો, We all deserve a better life. And it is just a decision away.  
Take care.  Love and gratitude.
-ડૉ. નિમિત્ત ઓઝાા"""

In [18]:
text="""અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું. એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા. સવારના પહોરમાં અખબાર વાંચતાની સાથે જ એ છોકરીને લાગેલું કે એનો બોયફ્રેન્ડ હવે એની સામે પણ નહીં જુએ. "છાપામાં છપાઈ ગયું..." એ કેટલી ભયાનક વાત હોઈ શકે, આજથી અઢી દાયકા પહેલાં ! સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો... સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં, એણે હસીને કહ્યું, "લખ્યા કરે... મારે માટે તું વધારે સાચી કે આ લખનાર ?"
આજથી અઢી દાયકા પહેલાં, એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?" તો એને માટે સન્માન થયા વગર રહે ? જોકે, ત્યારે સન્માનનો અર્થ પ્રેમ થાય કે નહીં, એ વિશે હજી નિર્ણય કરવાની મારી ઉંમર નહોતી અને મનઃસ્થિતિ પણ નહીં.
"એક વર્ષથી આપણે સાથે ફરીએ છીએ. મારા મા-બાપ રોજ પૂછે છે... હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું ?" ઠંડીના દિવસોમાં ગાંધીગ્રામ સ્ટેશન ઉપર, હાથમાં ચાના કપ પકડીને પસાર થતી ટ્રેનના અવાજમાં એણે મને પૂછ્યું હતું ! જો હું ના પાડું, તો ટ્રેનના અવાજમાં એ ના દબાઈ જાય... એવી એની ગણતરી હતી જે એણે મને પછીથી કહેલું !
          ત્યારે હા કે ના કહેવાનો સવાલ જ નહોતો... એની સાથે જીવવાની ઈચ્છા હતી, પણ લગ્ન વિશે મારા મનમાં બહુ મોટો અવઢવ હતો, કારણ એ, કે મારા પ્રથમ લગ્નમાં હું અપમાન, દાદાગીરી અને ડોમેસ્ટીક વાયોલેન્સનો ભોગ બની ચૂકી હતી. ઓગણીસ વર્ષે, જિંદગી શરૂ થાય એ પહેલાં એરેન્જ મેરેજ પૂરાં થઈ ચૂક્યાં હતાં ! હું કદાચ મારા સમયથી બહુ આગળ જીવી હોઈશ. આજના છોકરાંઓને કમિટમેન્ટનો વિરોધ કરતા જોઉં ત્યારે લાગે કે મારા મનમાં આ ગૂંચવણ 27 વર્ષ વહેલી જન્મી ગઈ હતી ! એના મનમાં ક્લેરીટી હતી. પરણવું છે ! ને, એ પરણ્યો... મારી સાથે! એના મિત્રો, પરિવારજનો અને બીજા ઘણાએ એને આવું નહીં કરવા સમજાવ્યો હતો, પણ એ સ્વભાવે સાહસિક છે... આજે પણ !
         નામ એનું સંજય વૈદ્ય. વ્યવસાયે ફોટોગ્રાફર, ને સ્વભાવે અલગારી. ત્યારે ગુજરાત સમાચારમાં નોકરી કરતો, પણ નોકરી એનો સ્વભાવ નથી. સાચું પૂછો તો એ લગ્ન કરવા માટેનું મટીરીયલ જ નથી. જોકે એ મિત્ર ખૂબ સારો. કલાકો સાંભળી શકે. આપણી છટકી હોય ત્યારે શાંત રહી શકે. ફોન ઉપર હું 'હલો' કહું ત્યાં જ મારા મૂડની એને ખબર પડે...
        એ મુક્ત વ્યક્તિ છે, સ્વભાવે, પ્રકૃતિએ, વિચારે અને વ્યવહારે પણ. એ બીજાને મુક્ત રાખવામાં માને છે, કારણ કે પોતે બંધાઈ શકે એમ નથી. એણે ક્યારેય મને "વૈદ્ય" ઉમેરવા વિશે, મારા વ્યવસાય વિશે કે આવક વિશે સવાલો કરવાની કે સલાહ આપવાની તસ્દી લીધી નથી. એ સીધા પ્રશ્નો ન પૂછે, એને મારા વિશે જાણવું કે કાળજી કરવી ગમે-પણ એની અનુકૂળતાએ. એ વ્યવહારુ નથી. લેવડ-દેવડ એને નથી આવડતી એમ કહી શકાય. એ બદમિજાજ નથી, ઓછાબોલો છે. બેદરકાર પણ નથી, બેફિકર ચોક્કસ છે. એની બેફિકરાઈ ક્યારેક આપણને ખૂંચે, કારણ કે આપણે એની પાસેથી અપેક્ષા રાખીએ... એ અપેક્ષા ન રાખે એટલે એને નિરાશા ઓછી જ થાય. 
         એકવાર એક પાર્ટીમાં એક બહેન બીજાને કહેતાં હતાં, "આ ભાઈ ઊભા છે ને એ કાજલબેનના 'એક્સ' હસબન્ડ છે." સંજયે એમના ખભે હાથ મૂકીને કહ્યું, "એમ ? તો કાજલે મને કેમ નથી કહ્યું ?" એની સેન્સ ઓફ હ્યુમર ક્યારેક સટલ છે તો ક્યારેક સ્ટુપીડ... બધાને સમજાતી નથી, ને કેટલાકને પચતી યે નથી ! એની સેન્સ ઓફ હ્યુમરનું ટાઈમિંગ ક્યારેક એટલું ખરાબ હોય કે આપણો ગુસ્સો ચાર ગણો વધી જાય, તો ક્યારેક ભયાનક ગુસ્સામાં હોઈએ ત્યારે એની વાત સાંભળીને હસવું આવી જાય. એ ઝઘડતા ઝઘડતા ભૂલી જાય કે અમે કઈ વાતે ઝઘડો શરૂ કર્યો હતો !
         અમે બંને સ્વભાવે અત્યંત મુક્ત, અને સ્વતંત્ર છીએ. ખાસ કરીને, મારો ઉછેર આજથી પચાસ વર્ષ પહેલાં, 'એક દીકરીનો થવો જોઈએ' એવો નથી થયો. મને નિર્ણય કરવાની, ભૂલો કરવાની, અથડાવા-કૂટાવાની, પરિણામ ભોગવવાની પૂર્ણ સ્વતંત્રતા મળી છે. ઘણા પુરુષો જે મારી સાથે વ્યવસાય કે મિત્રતાના સંબંધે સંકળાયા હોય એમને માટે આજે પણ હું "ઝૂમાંથી આવેલું પ્રાણી" છું. એમને હું સમજાતી નથી, સ્વીકારવાનો પ્રશ્ન તો પછી આવે છે ! 
        સંજય મને સમજે છે એટલું તો મારે સ્વીકારવું જ પડે. એણે મને સ્વતંત્રતા આપી છે, અલબત્ત પોતાની સ્વતંત્રતાની શરતે ! અમે બંને અત્યંત સારા મિત્રો છીએ. લડી શકીએ, ઝઘડી શકીએ, ખુલ્લા દિલે એકબીજાનો વિરોધ કરી શકીએ, એકબીજાની મજાક કરી શકીએ અને માતા-પિતા તરીકે પૂરી સમજણ અને કન્સર્નથી એકબીજાને સહકાર આપી શકીએ. અમારી પાસે ત્રણ ઘર છે, "મારું" કે "તારું" એવી વહેંચણીને બદલે અમે એ ઘરોને એડ્રેસથી ઓળખીએ છીએ. ચંદ્રકોલોની, ગાલા અને પલક... અમારા પરિવારના ચાર જણ, જેને જ્યાં ફાવે ત્યાં રહે છે, જ્યારે, જેમ અનુકૂળ હોય તેમ !
        અમે કદાચ આઈડિયલ કપલ નથી, અથવા અમે જ આઈડિયલ કપલ છીએ ! એક છત નીચે રહેવું એ દામ્પત્ય નથી, એક જ વિચાર, એક જ શોખ, એક જ સ્વાદ કે સતત એકબીજાને ગમતા રહેવું એ પણ દામ્પત્ય નથી જ... આવું અમે બંને માનીએ છીએ.
        હું ભાગ્યે જ એના વિશે વાત કરું છું, ને એ મારે વિશે વાત કરતો જ નથી. ઘણા લોકો માને છે કે અમારી વચ્ચે કોઈ વાંધો છે કે પછી અમારા લગ્ન તૂટી ગયાં છે. આજે, અમારા લગ્નને 27 વર્ષ પૂરાં થાય છે ત્યારે આ લખું છું. હું લીગલી, ઓફિસિયલી, સોશિયલી, ઈમોશનલી એની પત્ની છું.
લોકો શું માને છે એની પરવાહ મેં ક્યારેય કરી નથી, મારા સાસુ, મારો દીકરો, કે મારા મિત્રો શું માને છે એટલા પૂરતી મારી જવાબદારી છે અને રહેશે. મારા સસરા જીવ્યા ત્યાં સુધી મને 'કાજલ ગૌરી' કહેતા ! એમણે અમારા બંનેની આ સ્વતંત્રતાને પૂરા સ્નેહ અને સ્વીકારથી સન્માન આપ્યું હતું. 
        મારી મરણનોંધ 'કાજલ ઓઝા વૈદ્ય'ના નામે છપાય તો, અમારે વિશે કૂતુહલ ધરાવનારને નવાઈ ન લાગવી જોઈએ ! """

In [19]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize

keywords=['N-NNP','N_NN','JJ','N_NNV','N_NST','QT_QTC','QT_QTF','QT_QTO','PSP']
keywords1=['V_VAUX','RP_RPD','V_VM','V_VM_VF','VM_VM_VINF','V_VM_VNF','V_VM_VNG','PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ',
          'RB']
fc=[]
li=[]
tokenize_words=[]
tokenize_words1=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)
originalsen=[]
sentences=sentence_tokenize.sentence_split(text, lang='gu')
print(sentences)
for t in sentences:
    sen=removestopwords(t)
    tagged_words = (Combine_tagger.tag(nltk.word_tokenize(sen)))
            #originalsen.append(sen)
        
    lst1=[]
    for w in tagged_words:
        lst1.append(w[1])
    b = nltk.FreqDist()
    i=1
    j=1
    for w in lst1:
        if w in keywords:
            i=i+1

    for w1 in lst1:
        if w1 in keywords1:
            j=j+1

    f=(i-j+100)/2
    fc.append(f)
            #print(fc)
for i in range(len(fc)):
    li.append([fc[i],i])
li.sort(reverse=True)
        #print("this is li",li)
        
sort_index = []
for x in li:
    sort_index.append(x[1])
        #print(sort_index)
compressionratio = round((len(sort_index)*40)/100)
        #print("this is compression ratio",compressionratio)
newsen=[]
for j in range(compressionratio):
    newsen.append(sentences[sort_index[j]])
        #print("this is newsen",newsen)    
originalfull_str = convert_list_to_string(newsen)
originalfull_str=" ".join(originalfull_str.split())
    #n=str(n)
print("this is summary",originalfull_str)
fc.clear()
li.clear()
for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
    tokenize_words.append(tokenize)
        # Convert list of strings to string
originalfull_str = convert_list_to_string(tokenize_words)
senlateron=[]
sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
i=1
# data1="""વાઈવા આપતી વખતે, એપ્રનના પહેલા બે બટન ખોલી નાંખતી હશે.એમબીબીએસમાં મારી જ સાથે ભણતા એક મેડિકલ સ્ટુડન્ટે અમારા ક્લાસની ટોપર અને સૌથી હોંશિયાર છોકરી વિશે કરેલી ટિપ્પણી મને આજે પણ યાદ છે.
#     આપણી આસપાસ એવા અઢળક લોકો છે જે એવું માને છે કે સફળતાનો રસ્તો પુરુષ કરતા સ્ત્રી માટે વધારે ટૂંકો હોય છે.
#     સફળતાની સીડીઓ ચડી રહેલી સ્ત્રીને પાછળ ધકેલવા અને એણે મેળવેલી સિદ્ધિઓનો અનાદર કરવા માટે અન્ય કોઈ સ્ત્રી કે પુરુષ પાસે રહેલું  સૌથી  જુનું
#     અને ધારદાર  હથિયાર એટલે  'કેરેક્ટર એસેસીનેશન.'
#     ફેમિનિઝમના સહારે અને ભરોસે રહેવાને બદલે એક સ્ત્રી જ્યારે પોતાના હકની સિદ્ધિ અને સફળતા, પોતાની મહેનત અને ટેલેન્ટના
#     જોર પર ઝૂંટવી લે છે ત્યારે વાત એના ચારિત્ર્ય પર આવે છે.
#     ઈસ્ટ્રોજન સામેની દોડમાં જ્યારે ટેસ્ટોસ્ટેરોન ઘવાય છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે. જે સમયે કોઈપણ જાતના જજમેન્ટ
#     પાસ કર્યા વગર એક આગળ વધી રહેલી સ્ત્રીની સફળતાને આપણે 'એઝ ઈટ ઈઝ' સ્વીકારતા થઈ જશું, ત્યારે આપણે સિવિલાઈઝ્ડ 
#     કહેવાશું. ખ્યાતિ, લોકપ્રિયતા કે પ્રસિદ્ધિ મેળવવા માટે દરેક સ્ત્રીએ 'કોમ્પ્રોમાઈઝ' કરવું જ પડે છે,
#     એવું માનનારા લોકો પોતાની સમજણ સાથે કોમ્પ્રોમાઈઝ કરી રહ્યા છે."""

# data1="""All said and done, મને મળી રહેલા પર્સનલ મેસેજીસની સંખ્યા 'અનમેનેજબલ' થઈ રહી છે.મેઈલ કે મેસેજ દ્વારા જે રીતે અને
# જે સંખ્યામાં લોકો પોતાની ઉદાસી કબુલી રહ્યા છે, પોતાને બચાવવા માટે રીક્વેસ્ટ કરી રહ્યા છે.
#     ૧.સૌથી પહેલા તાત્કાલિક કોઈ થેરાપીસ્ટ, મનોચિકિત્સક કે કાઉન્સેલરને મળો.સારવારનો પહેલો તબક્કો સ્વીકાર છે.
#     ડિપ્રેશનની દવા અસર કરતા ૧૫-૨૦ દિવસ લાગે છે.
#     ત્યાં સુધી 'નો સ્યુસાઈડ કોન્ટ્રેક્ટ' સાઈન કરો.
# ૨.'મેન્ટલ ડાયેરીયા' બંધ કરવા માટે મગજને બિનજરૂરી અને વાહિયાત માહિતીઓ પીરસવાનું બંધ કરો.
# એન્ડ ઈન પ્રોસેસ ઓફ ડૂઇંગ સો, સૌથી પહેલા સોશિયલ મીડિયા અને ન્યુઝ ચેનલ્સ બંધ કરો.
# ફેસબુક, ઇન્સ્ટા, વ્હોટ્સ-એપ પરથી થોડો સમય બ્રેક લઈ લો.
# સોશિયલ મીડિયા એ ડિપ્રેશનનું સૌથી મોટું કારણ છે કારણકે Every time we sign into it, unknowingly we are landing into the zone
# of comparison.સંપર્ક માટેના ત્રણ શ્રેષ્ઠ રસ્તાઓ જ ખુલ્લા રાખો.
#     'રૂટીન પ્રાયોરીટી' કામ હોય તો ઈ-મેઈલ, સેમી-અરજન્ટ હોય તો ટેક્સ્ટ મેસેજ (SMS) અને બહુ અરજન્ટ હોય તો ફોન કોલ.
#     બ્રેક ન લો તો પણ, રીસર્ચ એવું કહે છે કે સોશિયલ મીડિયા પર ૩૦ મીનીટ કે ૧ કલાકથી વધારે સમય ન પસાર કરો.
# ૩.We all are constantly surrounded by people full of shit, and only Meditation can rescue us.
#     'ગાઇડેડ મેડિટેશન' માટે તમારા ફોનમાં 'Calm' કે 'Headspace' એપ ડાઉનલોડ કરો.
# 'https://www.thinklessandgrowrich.com/meditation-for-beginners-the-complete-guide/'.
# ૪.પુસ્તકો વાચો.Your brain deserves much better content than non-sense forwards, silly jokes, and good morning messages.
# Few good reads are : 1.Kick the thinking habit by Richard Paterson
# 2.The subtle art of not giving a F*ck  by Mark Manson
# 3.The power of Now by Eckhart Tolle 
# 4.Any book by Dalai Lama
# 5.Any book by Mahatma Gandhi
# 6.Any book on Buddha and Buddhism
# 7.વિનોબાજીના ગીતા પ્રવચનો
# ૫.Nature heals the soul.ક્યારેય ડિપ્રેશન નહીં આવે. હવે, તો આખી ફોરેસ્ટ મેડિસિનની બ્રાંચ ડેવલપ થઈ રહી છે, જે કહે છે કે 'If you hug a tree,
# It will make you happy.'
# ૬. મનમાં આવતા નેગેટીવ, નિરાશાજનક અને વાહિયાત વિચારોનું ઉત્સર્જન કરો.
# ડિપ્રેશન એ બીજું કાંઈ નથી પણ અવ્યક્ત રહી ગયેલા એટેન્શન સીકિંગ વિચારોનો આપણી બુદ્ધિ સામે થયેલો બળવો છે.
# ૭.ગ્રેટીટ્યુડ ટેકનીક : રોજ સવારે ઉઠીને સૌથી પહેલા એક કાગળ પર એવી ત્રણ વસ્તુઓ લખો, જેના માટે તમે ઈશ્વરના આભારી છો.
# એ કાંઈ પણ હોય શકે છે.
# આવું બધું વિચારશું, તો મરવાની ઈચ્છા નહીં થાય.
# ૮.એક્સરસાઈઝનો સૌથી મોટો ફાયદો માનસિક સ્વાસ્થ્ય પર થાય છે.
# જીમ કરો, સાયકલીંગ, રનીંગ, સ્કીપીંગ, સ્વીમીંગ કાંઈ પણ કરો.
# કાંઈ નહીં તો ગો ઓન અ લોંગ વોક.
# ૯.તમારા હેડ-ફોન્સ લગાડો અને તમારા ગમતા પ્લે-લીસ્ટમાં ખોવાઈ જાવ.
# ૧૦.ધ લીસ્ટ ઈઝ એન્ડ-લેસ. હજી ઘણું બાકી છે."""
data1=""" અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું.
    એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા.
     આજથી અઢી દાયકા પહેલાં ! સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો.
     સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં, એણે હસીને કહ્યું, "લખ્યા કરે... મારે માટે તું વધારે સાચી કે આ લખનાર ?"
આજથી અઢી દાયકા પહેલાં, એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?" તો એને માટે સન્માન થયા વગર રહે ? જોકે,
 એક વર્ષથી આપણે સાથે ફરીએ છીએ.
    હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું જો હું ના પાડું, તો ટ્રેનના અવાજમાં એ ના દબાઈ જાય.
    સાથે જીવવાની ઈચ્છા હતી,કારણ એ, કે મારા પ્રથમ લગ્નમાં હું અપમાન, દાદાગીરી અને ડોમેસ્ટીક.
    આજે પણ !નામ એનું સંજય વૈદ્ય.
    વ્યવસાયે ફોટોગ્રાફર, ને સ્વભાવે અલગારી.
    ત્યારે ગુજરાત સમાચારમાં નોકરી કરતો,
 એ સીધા પ્રશ્નો ન પૂછેએ વ્યવહારુ નથી.
    લેવડ-દેવડ એને નથી આવડતી એમ કહી શકાય.
    એ બદમિજાજ નથી, ઓછાબોલો છે.
    એની બેફિકરાઈ ક્યારેક આપણને ખૂંચે, કારણ કે આપણે એની પાસેથી અપેક્ષા રાખીએ.
    એ અપેક્ષા ન રાખે એટલે એને નિરાશા ઓછી જ થાય.
   એની સેન્સ ઓફ હ્યુમર ક્યારેક સટલ છે તો ક્યારેક સ્ટુપીડ!.
અમે બંને સ્વભાવે અત્યંત મુક્ત, અને સ્વતંત્ર છીએ.
    ખાસ કરીને, મારો ઉછેર આજથી પચાસ વર્ષ પહેલાં, "એક દીકરીનો થવો જોઈએ" એવો નથી થયો.
    ઘણા પુરુષો જે મારી સાથે વ્યવસાય કે મિત્રતાના સંબંધે સંકળાયા હોય એમને માટે આજે પણ હું "ઝૂમાંથી આવેલું પ્રાણી" છું.
    એમને હું સમજાતી નથી, સ્વીકારવાનો પ્રશ્ન તો પછી આવે છે ! સંજય મને સમજે છે એટલું તો મારે સ્વીકારવું જ પડે.
    અમે બંને અત્યંત સારા મિત્રો છીએ.
    અમારી પાસે ત્રણ ઘર છે, "મારું" કે "તારું" એવી વહેંચણીને બદલે અમે એ ઘરોને એડ્રેસથી ઓળખીએ છીએ.
    અમારા પરિવારના ચાર જણ, જેને જ્યાં ફાવે ત્યાં રહે છે,
    આજે, અમારા લગ્નને 27 વર્ષ પૂરાં થાય છે ત્યારે આ લખું છું.
    મારા સસરા જીવ્યા ત્યાં સુધી મને "કાજલ ગૌરી" કહેતા ! એમણે અમારા બંનેની આ સ્વતંત્રતાને પૂરા સ્નેહ અને સ્વીકારથી સન્માન આપ્યું હતું.
    મારી મરણનોંધ "કાજલ ઓઝા વૈદ્ય"ના નામે છપાય તો, અમારે વિશે કૂતુહલ ધરાવનારને નવાઈ ન લાગવી જોઈએ !"""
i=i+1
for tokenize1 in indic_tokenize.trivial_tokenize(data1):
        tokenize_words1.append(tokenize1)
        # Convert list of strings to string
originalfull_str1 = convert_list_to_string(tokenize_words1)
senlateron1=[]
sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
_, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
#rouge1.append(rouge_1)
print('ROUGE-1: %f' % rouge_1)
_, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
print('ROUGE-2: %f' % rouge_2)
#rouge2.append(rouge_2)
_, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-L: %f' % rouge_l)
#rougel.append(rouge_l)
_, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-W: %f' % rouge_w)
#rougew.append(rouge_w)

['અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું.', 'એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા.', 'સવારના પહોરમાં અખબાર વાંચતાની સાથે જ એ છોકરીને લાગેલું કે એનો બોયફ્રેન્ડ હવે એની સામે પણ નહીં જુએ.', '"છાપામાં છપાઈ ગયું. . . " એ કેટલી ભયાનક વાત હોઈ શકે, આજથી અઢી દાયકા પહેલાં !', 'સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો. . . સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં, એણે હસીને કહ્યું, "લખ્યા કરે.', ' . . મારે માટે તું વધારે સાચી કે આ લખનાર ?', '"\nઆજથી અઢી દાયકા પહેલાં, એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?', '" તો એને માટે સન્માન થયા વગર રહે ?', 'જોકે, ત્યારે સન્માનનો અર્થ પ્રેમ થાય કે નહીં, એ વિશે હજી નિર્ણય કરવાની મારી ઉંમર નહોતી અને મનઃસ્થિતિ પણ નહીં.', '"એક વર્ષથી આપણે સાથે ફરીએ છીએ.', 'મારા મા-બાપ રોજ પૂછે છે. . . હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું ?', '" ઠંડીના દિવસોમ

In [20]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize

keywords=['N-NNP','N_NN','JJ','N_NNV','N_NST','QT_QTC','QT_QTF','QT_QTO','PSP']
keywords1=['V_VAUX','RP_RPD','V_VM','V_VM_VF','VM_VM_VINF','V_VM_VNF','V_VM_VNG','PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ',
          'RB']
fc=[]
li=[]
tokenize_words=[]
tokenize_words1=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)
originalsen=[]
sentences=sentence_tokenize.sentence_split(text, lang='gu')
print(sentences)
for t in sentences:
    sen=removestopwords(t)
    tagged_words = (Combine_tagger.tag(nltk.word_tokenize(sen)))
            #originalsen.append(sen)
        
    lst1=[]
    for w in tagged_words:
        lst1.append(w[1])
    b = nltk.FreqDist()
    i=1
    j=1
    for w in lst1:
        if w in keywords:
            i=i+1

    for w1 in lst1:
        if w1 in keywords1:
            j=j+1

    f=(i-j+100)/2
    fc.append(f)
            #print(fc)
for i in range(len(fc)):
    li.append([fc[i],i])
li.sort(reverse=True)
        #print("this is li",li)
        
sort_index = []
for x in li:
    sort_index.append(x[1])
        #print(sort_index)
compressionratio = round((len(sort_index)*40)/100)
        #print("this is compression ratio",compressionratio)
newsen=[]
for j in range(compressionratio):
    newsen.append(sentences[sort_index[j]])
        #print("this is newsen",newsen)    
originalfull_str = convert_list_to_string(newsen)
originalfull_str=" ".join(originalfull_str.split())
    #n=str(n)
print("this is summary",originalfull_str)
fc.clear()
li.clear()
for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
    tokenize_words.append(tokenize)
        # Convert list of strings to string
originalfull_str = convert_list_to_string(tokenize_words)
senlateron=[]
sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
i=1
# data1="""વાઈવા આપતી વખતે, એપ્રનના પહેલા બે બટન ખોલી નાંખતી હશે.એમબીબીએસમાં મારી જ સાથે ભણતા એક મેડિકલ સ્ટુડન્ટે અમારા 
#       ક્લાસની ટોપર અને સૌથી હોંશિયાર છોકરી વિશે કરેલી ટિપ્પણી મને આજે પણ યાદ છે.પુરુષની સફળતાને એના સાહસ,
#       સમર્પણ અને મહેનત સાથે જોડવામાં આવે છે  જ્યારે  સ્ત્રીની સફળતાને શક્યતાઓ સાથે. એ કોઈ કલાકાર હોય, જર્નલીસ્ટકે 
#      ઓફિસ વર્કર. કોઈપણ ક્ષેત્રમાં એક સ્ત્રી જ્યારે અચાનક લાઈમ-લાઈટમાં આવે છે, ત્યારે એની પ્રસિદ્ધિનો યશ એની પ્રતિભા કરતા એની સુંદરતા, એના 
#      ફિઝીક અને એની 'ઉદારતા' ને  આપવામાં આવે છે.કારણકે ફેમિનિઝમ તો સ્ત્રીઓને સમાન હક અને તક અપાવવા માટે હોય છે.
#      ઈસ્ટ્રોજન સામેની દોડમાં જ્યારે ટેસ્ટોસ્ટેરોન ઘવાય છે, ત્યારે વાત એના ચારિત્ર્ય પર આવે છે.સ્ત્રીના સક્સેસને એની સેક્સ્યુઆલીટી સાથે જોડી દેવામાં આવે છે.
# સ્ત્રીની સફળતા એણે કરેલું સમાધાન નથી, એણે કરેલી સાધના છે.કુટુંબને આપેલા સમય સાથે, પોતાના મોજ-શોખ અને સુખ-સગવડો સાથે.
# એક સ્ત્રી જ્યારે પોતાના 'કમ્ફોર્ટ ઝોન' માંથી બહાર નીકળે છે, ત્યારે એની સફળતા બીજા અનેક લોકો માટે 'અનકમ્ફોર્ટેબલ' બની જતી હોય છે."""
# data1="""All said and done, મને મળી રહેલા પર્સનલ મેસેજીસની સંખ્યા 'અનમેનેજબલ' થઈ રહી છે.
# મેઈલ કે મેસેજ દ્વારા જે રીતે અને જે સંખ્યામાં લોકો પોતાની ઉદાસી કબુલી રહ્યા છે.
# એ જોઈને હું સ્તબ્ધ છું.So I thought, Let me share something from my personal experience.
# ૧.સૌથી પહેલા તાત્કાલિક કોઈ થેરાપીસ્ટ, મનોચિકિત્સક કે કાઉન્સેલરને મળો.સારવારનો પહેલો તબક્કો સ્વીકાર છે.ડિપ્રેશનની દવા અસર કરતા ૧૫-૨૦ દિવસ લાગે છે.
# ત્યાં સુધી 'નો સ્યુસાઈડ કોન્ટ્રેક્ટ' સાઈન કરો. 
# ૨.'મેન્ટલ ડાયેરીયા' બંધ કરવા માટે મગજને બિનજરૂરી અને વાહિયાત માહિતીઓ પીરસવાનું બંધ કરો.એન્ડ ઈન પ્રોસેસ ઓફ ડૂઇંગ સો, સૌથી પહેલા
# સોશિયલ મીડિયા અને ન્યુઝ ચેનલ્સ બંધ કરો.સોશિયલ મીડિયા એ ડિપ્રેશનનું સૌથી મોટું કારણ.
# ૩.મેડિટેશન.મેડિટેશન.મેડિટેશન.આજ નહીં તો કાલ નહીં તો વર્ષો પછી, મેડિટેશન સિવાય ઉદ્ધાર નથી.તમારે કોઈ શીબીર કે ધ્યાન ગુરુ પાસે જવાની જરૂર નથી.
# એ તમે ઘેર બેઠા કરી શકો છો અને એ પણ એકલા.
# 'ગાઇડેડ મેડિટેશન' માટે તમારા ફોનમાં Calm કે Headspace એપ ડાઉનલોડ કરો.
# હું હેડસ્પેસ યુઝ કરું છું એન્ડ આઈ એમ લવીંગ ઇટ.
# ૪.પુસ્તકો વાચો. Few good reads are : 
# 1.Kick the thinking habit by Richard Paterson 
# 2.The subtle art of not giving a F*ck  by Mark Manson 
# 3.The power of Now by Eckhart Tolle 
# 4.Any book by Dalai Lama
# 5. Any book by Mahatma Gandhi
# 6.Any book on Buddha and Buddhism
# 7.'વિનોબાજીના ગીતા પ્રવચનો' 

# ૫.ઈકો-થેરાપી : Nature heals the soul.
# 'If you hug a tree, It will make you happy.'  
# ૬.મનમાં આવતા નેગેટીવ, નિરાશાજનક અને વાહિયાત વિચારોનું ઉત્સર્જન કરો.
# ડિપ્રેશન એ બીજું કાંઈ નથી પણ અવ્યક્ત રહી ગયેલા એટેન્શન સીકિંગ વિચારોનો આપણી બુદ્ધિ સામે થયેલો બળવો છે.
# કેથાર્સીસ કે વિરેચન માટેનું કોઈ એક માધ્યમ શોધી કાઢો.
# ૭.ગ્રેટીટ્યુડ ટેકનીક : રોજ સવારે ઉઠીને સૌથી પહેલા એક કાગળ પર એવી ત્રણ વસ્તુઓ લખો, જેના માટે તમે ઈશ્વરના આભારી છો.
# એ કાંઈ પણ હોય શકે છે.જેમકે તમે જીવો છો. યુનિસેફના આંકડા પ્રમાણે વિશ્વના ૨૨,૦૦૦ બાળકો દરરોજ ભૂખમરાથી મૃત્યુ પામે છે.
# આવું બધું વિચારશું, તો મરવાની ઈચ્છા નહીં થાય.
# ૮.ભલે આળસ આવે, પણ પરાણે કસરત કરો.
# કાંઈ નહીં તો ગો ઓન અ લોંગ વોક.ધિસ વિલ મેક યુ ફીલ ગુડ.
# ૯.તમારા હેડ-ફોન્સ લગાડો અને તમારા ગમતા પ્લે-લીસ્ટમાં ખોવાઈ જાવ.
# ૧૦. ધ લીસ્ટ ઈઝ એન્ડ-લેસ.હજી ઘણું બાકી છે."""
data1="""અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું.
એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા.
"છાપામાં છપાઈ ગયું..." એ કેટલી ભયાનક વાત હોઈ શકે, આજથી અઢી દાયકા પહેલાં !
સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો... સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં,
એણે હસીને કહ્યું, "લખ્યા કરે... મારે માટે તું વધારે સાચી કે આ લખનાર ?"
એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?" 
"એક વર્ષથી આપણે સાથે ફરીએ છીએ.
મારા મા-બાપ રોજ પૂછે છે... હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું ?"  
જો હું ના પાડું, તો ટ્રેનના અવાજમાં એ ના દબાઈ જાય... લગ્ન વિશે મારા મનમાં બહુ મોટો અવઢવ હતો, 
કારણ એ, કે મારા પ્રથમ લગ્નમાં હું અપમાન, દાદાગીરી અને ડોમેસ્ટીક વાયોલેન્સનો ભોગ બની ચૂકી હતી. 
એના મનમાં ક્લેરીટી હતી.
પરણવું છે !
એ મુક્ત વ્યક્તિ છે, સ્વભાવે, પ્રકૃતિએ, વિચારે અને વ્યવહારે પણ.
એણે ક્યારેય મને "વૈદ્ય" ઉમેરવા વિશે, મારા વ્યવસાય વિશે કે આવક વિશે સવાલો કરવાની કે સલાહ આપવાની તસ્દી લીધી નથી.
લેવડ-દેવડ એને નથી આવડતી એમ કહી શકાય.
એની બેફિકરાઈ ક્યારેક આપણને ખૂંચે,કારણ કે આપણે એની પાસેથી અપેક્ષા રાખીએ... એ અપેક્ષા ન રાખે એટલે એને નિરાશા ઓછી જ થાય.
એની સેન્સ ઓફ હ્યુમરનું ટાઈમિંગ ક્યારેક એટલું ખરાબ હોય કે આપણો ગુસ્સો ચાર ગણો વધી જાય, તો ક્યારેક ભયાનક ગુસ્સામાં હોઈએ ત્યારે એની વાત 
સાંભળીને હસવું આવી જાય.
અમે બંને સ્વભાવે અત્યંત મુક્ત, અને સ્વતંત્ર છીએ.
મારો ઉછેર આજથી પચાસ વર્ષ પહેલાં, "એક દીકરીનો થવો જોઈએ" એવો નથી થયો.
ઘણા પુરુષો જે મારી સાથે વ્યવસાય કે મિત્રતાના સંબંધે સંકળાયા હોય એમને માટે આજે પણ હું "ઝૂમાંથી આવેલું પ્રાણી" છું.
અમે બંને અત્યંત સારા મિત્રો છીએ.
અમારી પાસે ત્રણ ઘર છે.
આજે, અમારા લગ્નને 27 વર્ષ પૂરાં થાય છે ત્યારે આ લખું છું.
મારા સાસુ, મારો દીકરો, કે મારા મિત્રો શું માને છે એટલા પૂરતી મારી જવાબદારી છે અને રહેશે."""
i=i+1
for tokenize1 in indic_tokenize.trivial_tokenize(data1):
        tokenize_words1.append(tokenize1)
        # Convert list of strings to string
originalfull_str1 = convert_list_to_string(tokenize_words1)
senlateron1=[]
sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
_, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
#rouge1.append(rouge_1)
print('ROUGE-1: %f' % rouge_1)
_, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
print('ROUGE-2: %f' % rouge_2)
#rouge2.append(rouge_2)
_, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-L: %f' % rouge_l)
#rougel.append(rouge_l)
_, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
print('ROUGE-W: %f' % rouge_w)
#rougew.append(rouge_w)

['અમદાવાદના એક જાણીતા અખબારમાં ચાર કોલમનું બોક્સ એક છોકરી વિશે છપાયેલું.', 'એના પિતા સાથેના કોઈક વાંધાને કારણે એના ચારિત્ર્ય ઉપર સાચા-ખોટા છાંટા ઉડાડવામાં આવેલા.', 'સવારના પહોરમાં અખબાર વાંચતાની સાથે જ એ છોકરીને લાગેલું કે એનો બોયફ્રેન્ડ હવે એની સામે પણ નહીં જુએ.', '"છાપામાં છપાઈ ગયું. . . " એ કેટલી ભયાનક વાત હોઈ શકે, આજથી અઢી દાયકા પહેલાં !', 'સવારના દસ વાગ્યા પહેલાં નહીં ઉઠનારો એનો બોયફ્રેન્ડ સાડા આઠ વાગ્યામાં ખમણ અને ગાંઠિયા લઈને એના ઘરે પહોંચ્યો. . . સહુએ સાથે નાસ્તો કર્યો એટલું જ નહીં, એણે હસીને કહ્યું, "લખ્યા કરે.', ' . . મારે માટે તું વધારે સાચી કે આ લખનાર ?', '"\nઆજથી અઢી દાયકા પહેલાં, એકવાર ડીવોર્સ થઈ ચૂક્યા હોય એવી છોકરી સાથે લગ્ન કરનારો કુંવારો છોકરો એને એકવાર પણ પૂછે નહીં, "એક્ચ્યુઅલી શું થયેલું ?', '" તો એને માટે સન્માન થયા વગર રહે ?', 'જોકે, ત્યારે સન્માનનો અર્થ પ્રેમ થાય કે નહીં, એ વિશે હજી નિર્ણય કરવાની મારી ઉંમર નહોતી અને મનઃસ્થિતિ પણ નહીં.', '"એક વર્ષથી આપણે સાથે ફરીએ છીએ.', 'મારા મા-બાપ રોજ પૂછે છે. . . હવે આપણે પરણી જવું જોઈએ એવું તને નથી લાગતું ?', '" ઠંડીના દિવસોમ